In [ ]:
import collections
import os

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import tqdm.notebook as tqdm
from sklearn.linear_model import LogisticRegression
from mordred import Calculator, descriptors
from rdkit import Chem
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import average_precision_score, \
    balanced_accuracy_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.model_selection import RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from correlation_threshold import CorrelationThreshold

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='sans-serif')
sns.set_palette(['#6da7de', '#9e0059', '#dee000', '#d82222', '#5ea15d',
                 '#943fa6', '#63c5b5', '#ff38ba', '#eb861e', '#ee266d'])
sns.set_context('paper', font_scale=1.3)

## Molecular descriptor feature generation

In [ ]:
compounds = pd.read_csv('../data/processed/compound_smiles.csv')
mols = compounds['smiles'].apply(Chem.MolFromSmiles)
clss = compounds['skin'].astype(np.int32)

In [ ]:
# Calculate features using Mordred.
mordred_calculator = Calculator(descriptors, ignore_3D=True)
# Exclude features that contain non-numeric values.
features = pd.DataFrame(mordred_calculator.pandas(mols, quiet=True)
                        .select_dtypes(exclude='object')
                        .astype(np.float32))

## Logistic regression training

In [ ]:
n_splits = 100
test_size = 0.2

variance_threshold = 0.05
correlation_threshold = 0.95

In [ ]:
# Repeatedly evaluate the model to estimate its performance.
accuracies_train = np.zeros(n_splits, np.float32)
accuracies_test = np.zeros(n_splits, np.float32)
average_precisions_train = np.zeros(n_splits, np.float32)
average_precisions_test = np.zeros(n_splits, np.float32)
roc_aucs_train = np.zeros(n_splits, np.float32)
roc_aucs_test = np.zeros(n_splits, np.float32)
interval = np.linspace(0, 1, 101, dtype=np.float32)
tprs_train = np.zeros((n_splits, 101), np.float32)
tprs_test = np.zeros((n_splits, 101), np.float32)
precisions_train = np.zeros((n_splits, 101), np.float32)
precisions_test = np.zeros((n_splits, 101), np.float32)
hyperparams = collections.defaultdict(list)
for i, (train_index, test_index) in tqdm.tqdm(
        enumerate(
            StratifiedShuffleSplit(
                n_splits, test_size=test_size, random_state=42).split(
                features.values, clss)),
        desc='Cross-validation', total=n_splits):
    features_train = features.values[train_index]
    features_test = features.values[test_index]
    clss_train, clss_test = clss[train_index], clss[test_index]
    
    hyperparam_search = RandomizedSearchCV(
        Pipeline([
            ('variance_threshold', VarianceThreshold(variance_threshold)),
            ('correlation_threshold', CorrelationThreshold(
                correlation_threshold)),
            ('scale', StandardScaler()),
            ('classify', LogisticRegression(
                penalty='elasticnet', random_state=42, solver='saga'))]),
        param_distributions={
            'classify__C': [0.001, 0.01, 0.1, 1, 10, 100],
            'classify__l1_ratio': np.arange(0., 1.01, 0.05)},
        n_iter=10,
        cv=StratifiedShuffleSplit(n_splits, test_size=test_size,
                                  random_state=42),
        random_state=42)
    
    hyperparam_search.fit(features_train, clss_train)
    pred_train = hyperparam_search.predict_proba(features_train)[:, 1]
    pred_test = hyperparam_search.predict_proba(features_test)[:, 1]
    
    # Compute evaluation metrics on the train and test data.
    accuracies_train[i] = balanced_accuracy_score(
            clss_train, np.asarray(pred_train > 0.5, np.int))
    accuracies_test[i] = balanced_accuracy_score(
            clss_test, np.asarray(pred_test > 0.5, np.int))
    average_precisions_train[i] = average_precision_score(
        clss_train, pred_train)
    average_precisions_test[i] = average_precision_score(
        clss_test, pred_test)
    roc_aucs_train[i] = roc_auc_score(clss_train, pred_train)
    roc_aucs_test[i] = roc_auc_score(clss_test, pred_test)
    fpr_train, tpr_train, _ = roc_curve(clss_train, pred_train)
    tprs_train[i] = np.interp(interval, fpr_train, tpr_train)
    fpr_test, tpr_test, _ = roc_curve(clss_test, pred_test)
    tprs_test[i] = np.interp(interval, fpr_test, tpr_test)
    precision_train, recall_train, _ = precision_recall_curve(
        clss_train, pred_train)
    precisions_train[i] = np.interp(
        interval, recall_train[::-1], precision_train[::-1])
    precision_test, recall_test, _ = precision_recall_curve(
        clss_test, pred_test)
    precisions_test[i] = np.interp(
        interval, recall_test[::-1], precision_test[::-1])
    
    # Save optimal hyperparameter(s).
    best_lr = hyperparam_search.best_estimator_.named_steps['classify']
    hyperparams['C'].append(best_lr.C)
    hyperparams['l1_ratio'].append(best_lr.l1_ratio)

In [ ]:
stats = {'accuracy_train': np.mean(accuracies_train),
         'accuracy_std_train': np.std(accuracies_train),
         'average_precision_train': np.mean(average_precisions_train),
         'average_precision_std_train': np.std(average_precisions_train),
         'roc_auc_train': np.mean(roc_aucs_train),
         'roc_auc_std_train': np.std(roc_aucs_train),
         'tpr_mean_train': np.mean(tprs_train, axis=0),
         'tpr_std_train': np.std(tprs_train, axis=0),
         'precision_mean_train': np.mean(precisions_train, axis=0),
         'precision_std_train': np.std(precisions_train, axis=0),
        
         'accuracy_test': np.mean(accuracies_test),
         'accuracy_std_test': np.std(accuracies_test),
         'average_precision_test': np.mean(average_precisions_test),
         'average_precision_std_test': np.std(average_precisions_test),
         'roc_auc_test': np.mean(roc_aucs_test),
         'roc_auc_std_test': np.std(roc_aucs_test),
         'tpr_mean_test': np.mean(tprs_test, axis=0),
         'tpr_std_test': np.std(tprs_test, axis=0),
         'precision_mean_test': np.mean(precisions_test, axis=0),
         'precision_std_test': np.std(precisions_test, axis=0)}
hyperparams = {key: scipy.stats.mode(val)[0][0]
               for key, val in hyperparams.items()}

In [ ]:
print(f'Accuracy: {stats["accuracy_test"]:.3f} ± '
      f'{stats["accuracy_std_test"]:.3f}')
print(f'AUROC: {stats["roc_auc_test"]:.3f} ± '
      f'{stats["roc_auc_std_test"]:.3f}')
print(f'Average precision: {stats["average_precision_test"]:.3f} ± '
      f'{stats["average_precision_std_test"]:.3f}')
print('Optimal hyperparameters:')
for key, val in hyperparams.items():
    print(f'    - {key} = {val}')

In [ ]:
# Train the optimal model on the entire dataset and export.
classifier = Pipeline([
    ('variance_threshold', VarianceThreshold(variance_threshold)),
    ('correlation_threshold', CorrelationThreshold(correlation_threshold)),
    ('scale', StandardScaler()),
    ('classify', LogisticRegression(
        penalty='elasticnet', random_state=42, solver='saga', **hyperparams))])
classifier.fit(features.values, clss)
_ = joblib.dump(classifier, '../data/processed/lr.joblib')

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

interval = np.linspace(0, 1, 101)
tpr_test = stats['tpr_mean_test']
tpr_test[0], tpr_test[-1] = 0, 1
ax.plot(interval, tpr_test,
        label=f'AUC = {stats["roc_auc_test"]:.3f} '
              f'± {stats["roc_auc_std_test"]:.3f}')
ax.fill_between(interval, tpr_test - stats['tpr_std_test'],
                tpr_test + stats['tpr_std_test'], alpha=0.2)
        
ax.plot([0, 1], [0, 1], c='black', ls='--')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('train_classifier_lr_roc.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))
        
precision_test = stats['precision_mean_test']
ax.plot(interval, precision_test,
        label=f'Avg precision = '
              f'{stats["average_precision_test"]:.3f} ± '
              f'{stats["average_precision_std_test"]:.3f}')
ax.fill_between(interval, precision_test - stats['precision_std_test'],
                precision_test + stats['precision_std_test'], alpha=0.2)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('train_classifier_lr_pr.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## Export

In [ ]:
_ = joblib.dump((stats, hyperparams), '../data/processed/train_classifier_lr.joblib')